In [5]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html


In [6]:
!pip uninstall jaxlib
#!pip install jaxlib==0.4.2
!pip install -U jaxlib==0.4.2+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.3.25+cuda11.cudnn805.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? Y
  Successfully uninstalled jaxlib-0.3.25+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.2%2Bcuda11.cudnn82-cp38-cp38-manylinux2014_x86_64.whl (164.4 MB)


In [1]:
# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
#%env JAX_PLATFORMS=cpu
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from utils import *
from layers import two_wl_aggregation

In [5]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [7]:
# for input "kernel_fn(x1, x2, 'ntk', pattern = (p1,p2))"
# the shape of the kernel after the convolution is:
# x1 = (b1, h1, w1, c1)
# x2 = (b2, h2, w2, c2)
# k = (b1, b2, h1, h2, w1, w2) 


L_branche = stax.serial(
    stax.Conv(100, (1,1)),
    )

Gamma_branche = stax.serial(
    stax.Conv(100, (1,1)),
    two_wl_aggregation(n_nodes=28),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    stax.Relu(),
    )


init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer, # k = (b1, b2, h1, h2, w1, w2)
    stax.GlobalSumPool(), # # k = (b1, b2)
)




In [8]:
# this needs ~9GB gpu-ram, takes ~15s on GPU ~2min on CPU (size 4 using XLA_PYTHON_CLIENT_PREALLOCATE=false)
%%time 
size = 20
r = 9
i = 0
j = 0

path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}"

x1 = np.load(path + f"/x1_{i*size}_{(1+i)*size}.npy")
x2 = np.load(path + f"/x2_{j*size}_{(1+j)*size}.npy")
p1 = np.load(path + f"/p1_{i*size}_{(1+i)*size}.npy")
p2 = np.load(path + f"/p2_{j*size}_{(1+j)*size}.npy")

kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))

XlaRuntimeError: ignored

In [9]:
import shutil
shutil.rmtree("/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/")

In [10]:
import os
newpath = "/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [8]:
# at some point this allways fails with 15gb gpu memory
# fails always for some function in utils
# can we make those function more memory efficient?

# with XLA_PYTHON_CLIENT_ALLOCATOR=platform this seems to stay below 5GB 
# GPU-RAM usage for size=4

# calculate kernel from saved preprocessed data
size = 10
r = 18
path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}" 

for i in range(1,r):
  print(i)
  for j in range(r):

    x1 = np.load(path + f"/x1_{i*size}_{(1+i)*size}.npy")
    x2 = np.load(path + f"/x2_{j*size}_{(1+j)*size}.npy")
    p1 = np.load(path + f"/p1_{i*size}_{(1+i)*size}.npy")
    p2 = np.load(path + f"/p2_{j*size}_{(1+j)*size}.npy")

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/kernel_matrix_{(1+i)*size}_{(1+j)*size}", kernel_matrix)

1


/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
